In [ ]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import regex as re


: 

In [ ]:
def fasta_reader(sequence_file):
    try:
        with open(sequence_file, 'r') as file:
            sequences = {}
            current_id = None
            current_seq = []

            for line in file:
                line = line.strip()
                if line.startswith('>'):
                    if current_id:
                        sequences[current_id] = ''.join(current_seq)
                    current_id = line[1:].replace(" ", "")
                    current_seq = []
                else:
                    current_seq.append(line)

            if current_id:
                sequences[current_id] = ''.join(current_seq)

            return list(sequences.items())[0]
    except FileNotFoundError:
        raise FileNotFoundError(f"Could not find file: {sequence_file}")

def validate_sequences(x, y):
    if not x or not y:
        raise ValueError("Empty sequences are not allowed")
    if not isinstance(x, str) or not isinstance(y, str):
        raise TypeError("Sequences must be strings")
    valid_chars = set('ACGT')  
    if not all(c in valid_chars for c in x + y):
        raise ValueError("Invalid sequence characters found")
    

file_1_name, file_1 = fasta_reader("human_gene.fna")
re = re.compile(r'\s+')
print(file_1_name)
print('\n\n')
print(file_1)

# **Needleman-Wunsch Global Sequence Alignment**

The **Needleman-Wunsch** algorithm is used for **global sequence alignment**, aligning the entire sequences of two strings. It uses dynamic programming to calculate the optimal alignment by considering **matches**, **mismatches**, and **gaps**.

---

### **Algorithm Overview**
1. **Initialization**: Create a scoring matrix with penalties for gaps and mismatches.
2. **Matrix Filling**: Compute the score for each cell based on the previous values and current characters.
3.

In [ ]:
# Implementing the Needleman-Wunsch global alignment
def nw(x, y, match=1, mismatch=1, gap=1):
    nx = len(x)
    ny = len(y)
    # Optimal score at each possible pair of characters.
    F = np.zeros((nx + 1, ny + 1))
    F[:, 0] = np.linspace(0, -nx * gap, nx + 1)
    F[0, :] = np.linspace(0, -ny * gap, ny + 1)
    # Pointers to trace through an optimal alignment.
    P = np.zeros((nx + 1, ny + 1))
    P[:, 0] = 3
    P[0, :] = 4
    # Temporary scores.
    t = np.zeros(3)
    for i in range(nx):
        for j in range(ny):
            if x[i] == y[j]:
                t[0] = F[i, j] + match 
            else:
                t[0] = F[i, j] - mismatch
            t[1] = F[i, j + 1] - gap # Downwards movement
            t[2] = F[i + 1, j] - gap # Rightwards movement
            tmax = np.max(t)
            F[i + 1, j + 1] = tmax
            if t[0] == tmax:
                P[i + 1, j + 1] += 2
            if t[1] == tmax:
                P[i + 1, j + 1] += 3
            if t[2] == tmax:
                P[i + 1, j + 1] += 4
    # Trace through an optimal alignment.
    i = nx
    j = ny
    rx = []
    ry = []
    while i > 0 or j > 0:
        if P[i, j] in [2, 5, 6, 9]:
            rx.append(x[i - 1])
            ry.append(y[j - 1])
            i -= 1
            j -= 1
        elif P[i, j] in [3, 5, 7, 9]:
            rx.append(x[i - 1])
            ry.append('-')
            i -= 1
        elif P[i, j] in [4, 6, 7, 9]:
            rx.append('-')
            ry.append(y[j - 1])
            j -= 1
    # Reverse the strings.
    rx = ''.join(rx)[::-1]
    ry = ''.join(ry)[::-1]

    # The final alignment score is the value in F[nx, ny]
    alignment_score = F[nx, ny]

    return '\n'.join([rx, ry]), alignment_score


---



# **Smith-Waterman Local Sequence Alignment**

The **Smith-Waterman** algorithm is used for **local sequence alignment**, which finds the most similar subsequences between two sequences. Unlike Needleman-Wunsch, it does not align the entire sequence but focuses on the best matching subsequence.

---

### **Algorithm Overview**
1. **Initialization**: Create a score matrix and initialize it to zero.
2. **Matrix Filling**: Compute scores for each cell, considering matches, mismatches, and gaps.
3. **Traceback**: Start from the cell with the highest score and trace the optimal local alignment path.


In [ ]:
def sw(x, y, match=1, mismatch=1, gap=1):
    nx = len(x)
    ny = len(y)
    # Create the score matrix
    H = np.zeros((nx + 1, ny + 1))
    max_score = 0
    max_pos = (0, 0)

    for i in range(1, nx + 1):
        for j in range(1, ny + 1):
            match_score = match if x[i - 1] == y[j - 1] else -mismatch
            H[i, j] = max(0, H[i - 1, j - 1] + match_score, H[i - 1, j] - gap, H[i, j - 1] - gap)

            if H[i, j] > max_score:
                max_score = H[i, j]
                max_pos = (i, j)

    # Traceback to get the optimal local alignment
    i, j = max_pos
    rx, ry = [], []
    while H[i, j] > 0:
        if H[i, j] == H[i - 1, j - 1] + (match if x[i - 1] == y[j - 1] else -mismatch):
            rx.append(x[i - 1])
            ry.append(y[j - 1])
            i -= 1
            j -= 1
        elif H[i, j] == H[i - 1, j] - gap:
            rx.append(x[i - 1])
            ry.append('-')
            i -= 1
        else:
            rx.append('-')
            ry.append(y[j - 1])
            j -= 1
    # Reverse the strings
    rx = ''.join(rx)[::-1]
    ry = ''.join(ry)[::-1]

    return rx, ry, max_score  # Returns both aligned sequences and score


In [ ]:
# Assuming the files are named as 'human_gene.fna', 'chimpanzee_gene.fna', and 'housemouse_gene.fna'
# Replace with the correct paths if needed

# Reading the sequences
file_1_name, file_1 = fasta_reader("human_gene.fna")
file_2_name, file_2 = fasta_reader("chimpanzee_gene.fna")
file_3_name, file_3 = fasta_reader("housemouse_gene.fna")

# Perform Needleman-Wunsch global alignment between all pairs
print("Needleman-Wunsch Global Alignment\n")

# Human vs Chimpanzee
output_1_nw, output_2_nw = nw(file_1, file_2)
print(f"Comparison: {file_1_name} vs {file_2_name}")
print(f"Gene ID: {file_1_name}")
print(f"Aligned Sequence: {output_1_nw}")
print(f"Gene ID: {file_2_name}")
print(f"Aligned Sequence: {output_2_nw}\n")

# Human vs House Mouse
output_1_nw, output_2_nw = nw(file_1, file_3)
print(f"Comparison: {file_1_name} vs {file_3_name}")
print(f"Gene ID: {file_1_name}")
print(f"Aligned Sequence: {output_1_nw}")
print(f"Gene ID: {file_3_name}")
print(f"Aligned Sequence: {output_2_nw}\n")

# Chimpanzee vs House Mouse
output_1_nw, output_2_nw = nw(file_2, file_3)
print(f"Comparison: {file_2_name} vs {file_3_name}")
print(f"Gene ID: {file_2_name}")
print(f"Aligned Sequence: {output_1_nw}")
print(f"Gene ID: {file_3_name}")
print(f"Aligned Sequence: {output_2_nw}\n")

# Perform Smith-Waterman local alignment between all pairs
print("Smith-Waterman Local Alignment\n")

# Human vs Chimpanzee
output_1_sw, output_2_sw, score_sw = sw(file_1, file_2)
print(f"Comparison: {file_1_name} vs {file_2_name}")
print(f"Gene ID: {file_1_name}")
print(f"Aligned Sequence: {output_1_sw}")
print(f"Gene ID: {file_2_name}")
print(f"Aligned Sequence: {output_2_sw}")
print(f"Local Alignment Score: {score_sw}\n")

# Human vs House Mouse
output_1_sw, output_2_sw, score_sw = sw(file_1, file_3)
print(f"Comparison: {file_1_name} vs {file_3_name}")
print(f"Gene ID: {file_1_name}")
print(f"Aligned Sequence: {output_1_sw}")
print(f"Gene ID: {file_3_name}")
print(f"Aligned Sequence: {output_2_sw}")
print(f"Local Alignment Score: {score_sw}\n")

# Chimpanzee vs House Mouse
output_1_sw, output_2_sw, score_sw = sw(file_2, file_3)
print(f"Comparison: {file_2_name} vs {file_3_name}")
print(f"Gene ID: {file_2_name}")
print(f"Aligned Sequence: {output_1_sw}")
print(f"Gene ID: {file_3_name}")
print(f"Aligned Sequence: {output_2_sw}")
print(f"Local Alignment Score: {score_sw}\n")


In [ ]:
# Sample function to compute pairwise alignment scores for clustering
def get_alignment_scores(sequences):
    scores = []
    for i in range(len(sequences)):
        for j in range(i + 1, len(sequences)):
            _, score_nw = nw(sequences[i], sequences[j])
            _, _, score_sw = sw(sequences[i], sequences[j])
            scores.append([score_nw, score_sw])
    return np.array(scores)

# Reading sequences from files
file_1_name, file_1 = fasta_reader("human_gene.fna")
file_2_name, file_2 = fasta_reader("chimpanzee_gene.fna")
file_3_name, file_3 = fasta_reader("housemouse_gene.fna")
file_4_name, file_4 = fasta_reader("whale_gene.fna")
file_5_name, file_5 = fasta_reader("sealion_gene.fna")
file_6_name, file_6 = fasta_reader("lion_gene.fna")
file_7_name, file_7 = fasta_reader("elephant_gene.fna")
file_8_name, file_8 = fasta_reader("frog_gene.fna")
file_9_name, file_9 = fasta_reader("snake_gene.fna")
file_10_name, file_10 = fasta_reader("cat_gene.fna")

sequences = [file_1, file_2, file_3, file_4, file_5, file_6, file_7, file_8, file_9, file_10]
sequence_names = [file_1_name, file_2_name, file_3_name, file_4_name, file_5_name, file_6_name, 
                  file_7_name, file_8_name, file_9_name, file_10_name]

# Compute alignment scores and labels for each data point (pair)
alignment_scores= get_alignment_scores(sequences)

# Standardize the alignment scores
scaler = StandardScaler()
scaled_scores = scaler.fit_transform(alignment_scores)

# Determine the optimal number of clusters using the Elbow Method
inertia = []
K = range(1, 10)  
for k in K:
    kmeans = KMeans(n_clusters=k, random_state=0, n_init='auto')  
    kmeans.fit(scaled_scores)
    inertia.append(kmeans.inertia_)

# Plot the Elbow Method to determine optimal clusters
plt.figure(figsize=(8, 5))
plt.plot(K, inertia, 'bo-')
plt.xlabel('Number of Clusters')
plt.ylabel('Inertia')
plt.title('Elbow Method for Optimal k')
plt.show()

# After choosing an optimal k (e.g., 3), perform clustering
optimal_k = 3  # Adjust based on Elbow plot results
kmeans = KMeans(n_clusters=optimal_k, random_state=0, n_init='auto')
kmeans.fit(scaled_scores)
labels = kmeans.labels_

print("K-means Clustering Results")
for i, (name, label) in enumerate(zip(sequence_names, labels)):
    print(f"Sequence: {name}, Cluster: {label}")

# Plotting the clusters with labels for each data point
plt.figure(figsize=(8, 6))
scatter = plt.scatter(scaled_scores[:, 0], scaled_scores[:, 1], c=labels, cmap='viridis')
plt.xlabel('Standardized Needleman-Wunsch Score')
plt.ylabel('Standardized Smith-Waterman Score')
plt.title(f'Clustering of Gene Sequences (k={optimal_k})')
plt.colorbar(scatter, label='Cluster')
plt.show()